In [119]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [120]:
#reading the dataset
zomato_real=pd.read_csv("./zomato.csv")
zomato_real.head() 

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [121]:
zomato_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [122]:
#Deleting Unnnecessary Columns
zomato=zomato_real.drop(['url','dish_liked','phone'],axis=1) 
#Dropping the column "dish_liked", "phone", "url" and saving the new dataset as "zomato"

In [123]:
print("Duplicate Rows:", zomato.duplicated().sum())  # Count duplicates
zomato.drop_duplicates(inplace=True)  # Remove duplicates
print("Duplicate Rows After Removal:", zomato.duplicated().sum())  # Verify


Duplicate Rows: 43
Duplicate Rows After Removal: 0


In [124]:
# Count and display missing values before removal
print("Missing Values Before Removal:\n", zomato.isnull().sum())

# Remove rows with NaN values
zomato.dropna(how='any', inplace=True)

# Display missing values after removal
print("\nMissing Values After Removal:\n", zomato.isnull().sum())

# Print dataset summary
print("\nDataset Summary After NaN Removal:")
print(zomato.info())


Missing Values Before Removal:
 address                           0
name                              0
online_order                      0
book_table                        0
rate                           7767
votes                             0
location                         21
rest_type                       227
cuisines                         45
approx_cost(for two people)     345
reviews_list                      0
menu_item                         0
listed_in(type)                   0
listed_in(city)                   0
dtype: int64

Missing Values After Removal:
 address                        0
name                           0
online_order                   0
book_table                     0
rate                           0
votes                          0
location                       0
rest_type                      0
cuisines                       0
approx_cost(for two people)    0
reviews_list                   0
menu_item                      0
listed_in(type)        

In [125]:
#Reading Column Names
zomato.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'approx_cost(for two people)',
       'reviews_list', 'menu_item', 'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [126]:
#Changing the column names
zomato = zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type',
                                  'listed_in(city)':'city'})
zomato.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city'],
      dtype='object')

In [127]:
#Some Transformations
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost to '.' eg 1,500 to 1.500
zomato['cost'] = zomato['cost'].astype(float) # Changing the cost to Float now it converted from 1.500->1500.0
zomato.info()


<class 'pandas.core.frame.DataFrame'>
Index: 43499 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address       43499 non-null  object 
 1   name          43499 non-null  object 
 2   online_order  43499 non-null  object 
 3   book_table    43499 non-null  object 
 4   rate          43499 non-null  object 
 5   votes         43499 non-null  int64  
 6   location      43499 non-null  object 
 7   rest_type     43499 non-null  object 
 8   cuisines      43499 non-null  object 
 9   cost          43499 non-null  float64
 10  reviews_list  43499 non-null  object 
 11  menu_item     43499 non-null  object 
 12  type          43499 non-null  object 
 13  city          43499 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 5.0+ MB


In [128]:
zomato['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [129]:
# Removing '/5' from Rates

# Removing rows where 'rate' is 'NEW'
zomato = zomato.loc[zomato.rate != 'NEW']

# Removing rows where 'rate' is '-' and resetting index
zomato = zomato.loc[zomato.rate != '-'].reset_index(drop=True)

# Defining a lambda function to remove '/5' from ratings
remove_slash = lambda x: x.replace('/5', '') if isinstance(x, str) else x

# Applying the lambda function to 'rate' column, stripping spaces, and converting to float
zomato.rate = zomato.rate.apply(remove_slash).str.strip().astype('float')

# Displaying first few values of 'rate' after transformation
zomato['rate'].head()


0    4.1
1    4.1
2    3.8
3    3.7
4    3.8
Name: rate, dtype: float64

In [130]:
# Adjust the column names

# Capitalizing the first letter of each word in the 'name' column
zomato.name = zomato.name.apply(lambda x: x.title())

# Replacing 'Yes' with True and 'No' with False in the 'online_order' column
zomato.online_order.replace(('Yes', 'No'), (True, False), inplace=True)

# Replacing 'Yes' with True and 'No' with False in the 'book_table' column
zomato.book_table.replace(('Yes', 'No'), (True, False), inplace=True)

# Displaying unique values in the 'cost' column to check the range of different costs
zomato.cost.unique()


array([800.  , 300.  , 600.  , 700.  , 550.  , 500.  , 450.  , 650.  ,
       400.  , 900.  , 200.  , 750.  , 150.  , 850.  , 100.  ,   1.2 ,
       350.  , 250.  , 950.  ,   1.  ,   1.5 ,   1.3 , 199.  ,   1.1 ,
         1.6 , 230.  , 130.  ,   1.7 ,   1.35,   2.2 ,   1.4 ,   2.  ,
         1.8 ,   1.9 , 180.  , 330.  ,   2.5 ,   2.1 ,   3.  ,   2.8 ,
         3.4 ,  50.  ,  40.  ,   1.25,   3.5 ,   4.  ,   2.4 ,   2.6 ,
         1.45,  70.  ,   3.2 , 240.  ,   6.  ,   1.05,   2.3 ,   4.1 ,
       120.  ,   5.  ,   3.7 ,   1.65,   2.7 ,   4.5 ,  80.  ])

In [131]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [132]:
zomato['city'].unique() #retrieves and displays all unique values in the 'city' column of the zomato DataFrame.


array(['Banashankari', 'Bannerghatta Road', 'Basavanagudi', 'Bellandur',
       'Brigade Road', 'Brookefield', 'BTM', 'Church Street',
       'Electronic City', 'Frazer Town', 'HSR', 'Indiranagar',
       'Jayanagar', 'JP Nagar', 'Kalyan Nagar', 'Kammanahalli',
       'Koramangala 4th Block', 'Koramangala 5th Block',
       'Koramangala 6th Block', 'Koramangala 7th Block', 'Lavelle Road',
       'Malleshwaram', 'Marathahalli', 'MG Road', 'New BEL Road',
       'Old Airport Road', 'Rajajinagar', 'Residency Road',
       'Sarjapur Road', 'Whitefield'], dtype=object)

In [133]:
## Checking Null values
zomato.isnull().sum()

address         0
name            0
online_order    0
book_table      0
rate            0
votes           0
location        0
rest_type       0
cuisines        0
cost            0
reviews_list    0
menu_item       0
type            0
city            0
dtype: int64

In [134]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [135]:
# Count the number of duplicate restaurant names
duplicate_counts = zomato.duplicated(subset=['name'], keep=False).sum()
print(f"Number of duplicate restaurant names: {duplicate_counts}")

# Get a list of unique restaurant names
restaurants = list(zomato['name'].unique())

# Create a new column 'Mean Rating' and initialize it with 0
zomato['Mean Rating'] = 0

# Iterate over each unique restaurant name
for i in range(len(restaurants)):
    # Compute the mean rating for each restaurant and assign it
    zomato.loc[zomato['name'] == restaurants[i], 'Mean Rating'] = zomato.loc[zomato['name'] == restaurants[i], 'rate'].mean()

# Log the completion message
print("Mean rating calculation completed!")


Number of duplicate restaurant names: 40559
Mean rating calculation completed!


In [136]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.118182
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.100000
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.800000
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.700000
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.800000


In [137]:
# Importing MinMaxScaler from scikit-learn
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler is used for feature scaling (normalization).
# It transforms values into a specified range (default: 0 to 1).
# Here, we set the range between 1 and 5.
scaler = MinMaxScaler(feature_range=(1, 5))

# Applying MinMaxScaler on the 'Mean Rating' column to scale its values between 1 and 5.
# .fit_transform() fits the data and scales it in one step.
# .round(2) ensures values are rounded to 2 decimal places.
zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)

# Displaying a random sample of 3 rows from the DataFrame to check the transformed values.
zomato.sample(3)


,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
24321,"11, 80 Feet Main Road, Opposite Indian Oil Pet...",The Sweet Spot,False,False,4.0,157,Koramangala 6th Block,Dessert Parlor,Desserts,300.0,"[('Rated 4.0', ""RATED\n Firstly this place is...",[],Delivery,Koramangala 5th Block,3.84
6062,"Poornima Complex, Besides South India Mall, Wh...",Master Brothers,True,False,3.3,7,Whitefield,Bakery,"Desserts, Bakery",450.0,"[('Rated 4.0', 'RATED\n Highly recommended. G...","['Black Forest Cake [500 Grams]', 'Black Curre...",Delivery,Brookefield,3.02
41166,"7, Anurhh Arcade, Hoodi Road, Kodigehalli Road...",Thai & Chinese Fast Food,False,False,3.4,7,KR Puram,Quick Bites,"Chinese, Thai",400.0,"[('Rated 5.0', 'RATED\n Its Looking delicious...",[],Dine-out,Whitefield,3.06


In [138]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,3.99
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,3.97
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.58
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.45
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.58


In [139]:
# 5 examples of these columns before text processing:
zomato[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
12571,"[('Rated 5.0', 'RATED\n just great !! it was ...","North Indian, Chinese, Biryani"
15588,"[('Rated 5.0', 'RATED\n Truly amazed and deli...","American, Chinese, Continental, North Indian"
19472,"[('Rated 4.0', ""RATED\n Food is really fresh ...","Healthy Food, North Indian, Biryani, Continent..."
6932,"[('Rated 5.0', 'RATED\n The parathas are the ...","South Indian, North Indian"
3496,"[('Rated 3.5', ""RATED\n Had a fantastic purch...","Bakery, Desserts"


In [140]:
## Lower Casing
zomato["reviews_list"] = zomato["reviews_list"].str.lower()
zomato[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
37756,"[('rated 3.0', 'rated\n amazing ambience. pre...","Cafe, Italian"
13960,"[('rated 1.0', 'rated\n unfortunately, this i...",North Indian
24694,"[('rated 1.0', 'rated\n ordered the grilled c...","Juices, Fast Food"
13613,"[('rated 1.0', 'rated\n extra sugar in the ju...","Juices, South Indian, Chinese, Fast Food"
7176,"[('rated 5.0', 'rated\n highly recommended fo...","Beverages, Juices, Healthy Food"


In [141]:
## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation #!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ provided letters 
# str.translate(x,y,z) 
# x: Characters to replace (not used in our case).
# y: Characters to replace them with (not used in our case).
# z: Characters to remove (we use this to remove punctuation).
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))
zomato[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
9022,rated 10 ratedn pathetic momos i ordered frie...,"Chinese, Fast Food, Momos"
34859,rated 40 ratedn had been there for a team lun...,"Chinese, Thai, Indonesian"
23692,rated 30 ratedn after looking at the reviews ...,North Indian
9156,rated 50 ratedn must try pub in koramangalawi...,"Continental, North Indian, Chinese"
28680,rated 50 ratedn we had 2 ghee rice1 butter ch...,"North Indian, Chinese, Biryani, Beverages"


In [40]:
# Stopwords are commonly used words (like "is," "the," "in," "and") that do not add much meaning to the text.
# Removing stopwords helps in improving text analysis and machine learning models (e.g., sentiment analysis, search optimization).
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Deep/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [142]:
## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))

In [143]:
## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_urls(text))

In [144]:
zomato[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
4335,rated 40 ratedn cute cozy cafe indiranagar twi...,"Cafe, Bakery"
11469,rated 15 ratedn worth money games operational ...,North Indian
13200,rated 10 ratedn worst pizza ordered zomoto bad...,Continental
5309,rated 40 ratedn budget friendly place pitcher ...,"North Indian, Chinese"
31379,rated 40 ratedn great place days craving indul...,Ice Cream


In [145]:
# RESTAURANT NAMES:
restaurant_names = list(zomato['name'].unique())
restaurant_names

['Jalsa',
 'Spice Elephant',
 'San Churro Cafe',
 'Addhuri Udupi Bhojana',
 'Grand Village',
 'Timepass Dinner',
 'Rosewood International Hotel - Bar & Restaurant',
 'Onesta',
 'Penthouse Cafe',
 'Smacznego',
 'Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â© Down The Alley',
 'Cafe Shuffle',
 'The Coffee Shack',
 'Caf-Eleven',
 'Cafe Vivacity',
 'Catch-Up-Ino',
 "Kirthi'S Biryani",
 'T3H Cafe',
 '360 Atoms Restaurant And Cafe',
 'The Vintage Cafe',
 'Woodee Pizza',
 'Cafe Coffee Day',
 'My Tea House',
 'Hide Out Cafe',
 'Cafe Nova',
 'Coffee Tindi',
 'Sea Green Cafe',
 'Cuppa',
 "Srinathji'S Cafe",
 'Redberrys',
 'Foodiction',
 'Sweet Truth',
 'Ovenstory Pizza',
 'Faasos',
 'Behrouz Biryani',
 'Fast And Fresh',
 'Szechuan Dragon',
 'Empire Restaurant',
 'Maruthi Davangere Benne Dosa',
 'Chaatimes',
 'Havyaka Mess',
 "Mcdonald'S",
 "Domino'S Pizza",
 'Hotboxit',
 'Kitchen Garden',
 'Recipe',
 'Beijing Bites',
 'Tasty Bytes',
 'Petoo',
 'Shree Cool Point'

In [148]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_words(column, top_nu_of_words, nu_of_word):
    """
    Extracts the top N most frequent words or n-grams from the given text column.
    
    Parameters:
    - column: list or pandas Series containing text data
    - top_nu_of_words: Number of top words to return
    - nu_of_word: Tuple specifying the n-gram range (e.g., (1,1) for unigrams, (2,2) for bigrams)
    
    Returns:
    - List of tuples containing the top N words/n-grams and their frequencies
    """

    vec = CountVectorizer(ngram_range=nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

    return words_freq[:top_nu_of_words]


In [149]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,[],Buffet,Banashankari,3.99
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,rated 40 ratedn dinner family turned good choo...,[],Buffet,Banashankari,3.97
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,rated 30 ratedn ambience good enough pocket fr...,[],Buffet,Banashankari,3.58
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,rated 40 ratedn great food proper karnataka st...,[],Buffet,Banashankari,3.45
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,rated 40 ratedn good restaurant neighbourhood ...,[],Buffet,Banashankari,3.58


In [150]:
zomato.sample(5)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
15387,"3, Christ School Main Road, Bharathi Layout, S...",Black Cup Cafe,True,False,3.6,19,BTM,Cafe,"Cafe, Fast Food, Beverages",400.0,rated 50 ratedn best place jamgood food great ...,[],Cafes,Jayanagar,3.42
4871,"4, KH Road, Double Road, Shanti Nagar, Bangalore",The Bangalore Cafe,False,True,4.1,473,Shanti Nagar,Casual Dining,"Modern Indian, North Indian, Pizza, Continenta...",800.0,rated 50 ratedn probably best pocket friendly ...,[],Dine-out,Brigade Road,4.07
28121,"2nd Cross, Venkateshwara Layout, Tavarekere Ma...",Sri Krishna Kitchen,True,False,3.6,42,BTM,Quick Bites,Fast Food,400.0,rated 20 ratedn place known unlimited thali ge...,"['Gobi Manchurian', 'Veg Fried Rice', 'Chicken...",Delivery,Koramangala 7th Block,3.34
18421,"24th Main Road, 5th Phase, JP Nagar, Bangalore",Empire Restaurant,True,False,4.1,2402,JP Nagar,Casual Dining,"North Indian, Mughlai, South Indian, Chinese",750.0,rated 20 ratedn first time came 4 months servi...,"['Empire Special Chicken Kebab', 'Chilly Chick...",Dine-out,JP Nagar,3.88
30757,"3, Rest House Road, Off Brigade Road, Brigade ...",Le Rock,True,False,4.1,1214,Brigade Road,Bar,"Continental, Chinese",2.0,rated 40 ratedn food 55ndrinks 45nambiance 35n...,"['Chicken Wings BBQ Sauce', 'Chicken Classic C...",Dine-out,Lavelle Road,3.89


In [151]:
zomato=zomato.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)

In [152]:
zomato.columns

Index(['name', 'online_order', 'book_table', 'rate', 'location', 'cuisines',
       'cost', 'reviews_list', 'city', 'Mean Rating'],
      dtype='object')

In [153]:
import pandas

# Randomly sample 50% of your dataframe
df_percent = zomato.sample(frac=0.5)

In [154]:
df_percent.shape

(20618, 10)

In [155]:
df_percent.set_index('name', inplace=True)

In [156]:
indices = pd.Series(df_percent.index)

In [157]:
# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0.000001, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

In [158]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [104]:
print(cosine_similarities)

[[1.         0.09830126 0.02383782 ... 0.2905258  0.0287661  0.06699998]
 [0.09830126 1.         0.12500637 ... 0.09510618 0.14018904 0.07926963]
 [0.02383782 0.12500637 1.         ... 0.01929915 0.06774964 0.05034757]
 ...
 [0.2905258  0.09510618 0.01929915 ... 1.         0.02113834 0.08723329]
 [0.0287661  0.14018904 0.06774964 ... 0.02113834 1.         0.04209005]
 [0.06699998 0.07926963 0.05034757 ... 0.08723329 0.04209005 1.        ]]


In [159]:
def recommend(name, cosine_similarities=cosine_similarities):
    # Create a list to store top recommended restaurants
    recommend_restaurant = []
    
    # Find the index of the restaurant entered
    idx = indices[indices == name].index[0]
    
    # Find restaurants with similar cosine similarity values and sort them in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract the top 30 restaurant indexes with similar cosine similarity values
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Get the names of these top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Create a new DataFrame to store similar restaurant details
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Fetch details of the top 30 similar restaurants
    for each in recommend_restaurant:
        df_new = pd.concat([df_new, df_percent[['cuisines', 'Mean Rating', 'cost']][df_percent.index == each].sample()])
    
    # Drop duplicate restaurants and select the top 10 based on rating
    df_new = df_new.drop_duplicates(subset=['cuisines', 'Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print(f'TOP {len(df_new)} RESTAURANTS LIKE {name} WITH SIMILAR REVIEWS:')
    
    return df_new


In [160]:
# HERE IS A RANDOM RESTAURANT. LET'S SEE THE DETAILS ABOUT THIS RESTAURANT:
df_percent[df_percent.index == 'Pai Vihar'].head()

,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
name,,,,,,,,,
Pai Vihar,True,False,2.8,Vasanth Nagar,"South Indian, Street Food, Chinese, Fast Food",400.0,rated 30 ratedn 12 rate hereãx83ãx83ãx82ãx...,Malleshwaram,2.48
Pai Vihar,False,False,3.2,City Market,"South Indian, Street Food, Chinese, Fast Food",400.0,rated 20 ratedn food dry bland dont understand...,Church Street,2.48
Pai Vihar,True,False,2.8,Vasanth Nagar,"South Indian, Street Food, Chinese, Fast Food",400.0,rated 30 ratedn 12 rate herenneven though tast...,Frazer Town,2.48
Pai Vihar,True,False,2.8,Vasanth Nagar,"South Indian, Street Food, Chinese, Fast Food",400.0,rated 30 ratedn 12 rate herenneven though tast...,Church Street,2.48
Pai Vihar,True,False,2.8,Vasanth Nagar,"South Indian, Street Food, Chinese, Fast Food",400.0,rated 30 ratedn 12 rate hereãx83ãx83ãx82ãx...,MG Road,2.48


In [161]:
recommend('The Marash')

TOP 8 RESTAURANTS LIKE The Marash WITH SIMILAR REVIEWS:


,cuisines,Mean Rating,cost
Milano Ice Cream,"Ice Cream, Desserts",4.61,400.0
Corner House Ice Cream,"Ice Cream, Desserts",4.44,400.0
Stoner,"Ice Cream, Cafe, Beverages, Burger, Desserts, ...",4.39,550.0
Icebreakers,"Desserts, Ice Cream",4.08,400.0
Oh So Stoned!,"Beverages, Desserts",3.97,250.0
Ice Cream Works,"Desserts, Ice Cream",3.79,300.0
Ibaco,"Desserts, Ice Cream",3.77,300.0
Mist N Creams,Ice Cream,3.41,400.0


In [105]:
!jupyter nbconvert --to script your_notebook.ipynb

zsh:1: command not found: jupyter
